# GR6J example

In [1]:
from pathlib import Path
import pandas as pd
import datetime
from hydrogr import InputDataHandler, ModelGr6j
import plotly.graph_objects as go

## Prepare the input data

Here we load a dataframe that contains daily rainfall and evapotranspiration data. The data is already daily sampled.
For demonstration purpose, we will not run the model on all the dataset period, but on a sub-period from 1989 to 1999.

In [2]:
data_path = Path.cwd().parent / 'data'
df = pd.read_pickle(data_path / 'L0123001.pkl')
df.columns = ['date', 'precipitation', 'temperature', 'evapotranspiration', 'flow', 'flow_mm']
df.index = df['date']

inputs = InputDataHandler(ModelGr6j, df)
start_date = datetime.datetime(1989, 1, 1, 0, 0)
end_date = datetime.datetime(1999, 12, 31, 0, 0)
inputs = inputs.get_sub_period(start_date, end_date)

## Setup the model

Here we define the model parameters and the initial states.
Model parameters are mandatory at model creation, and can be changed afterward using the set_parameters method.

In [3]:
# Set the model :
parameters = {
        "X1": 242.257,
        "X2": 0.637,
        "X3": 53.517,
        "X4": 2.218,
        "X5": 0.424,
        "X6": 4.759
    }
model = ModelGr6j(parameters)
model.set_parameters(parameters)  # Re-define the parameters for demonstration purpose.

# Initial state :
initial_states = {
    "production_store": 0.5,
    "routing_store": 0.6,
    "exponential_store": 0.3,
    "uh1": None,
    "uh2": None
}
model.set_states(initial_states)

## Run the model and display the results

In [4]:
outputs = model.run(inputs.data)
print(outputs.head())

                flow
date                
1989-01-01  5.370637
1989-01-02  3.786042
1989-01-03  3.003636
1989-01-04  2.357846
1989-01-05  1.952775


In [5]:
fig = go.Figure([
    go.Scatter(x=outputs.index, y=outputs['flow'], name="Calculated"),
    go.Scatter(x=inputs.data.index, y=inputs.data['flow_mm'], name="Observed"),
])
fig.show()